In [ ]:
# import os → lets you interact with the operating system (e.g., read environment variables, list files).
import os
# import json → allows working with JSON (JavaScript Object Notation) data, which is often used for APIs.
import json
# loads the Pandas library (used for data analysis, reading CSV/Excel).
import pandas as pd
# import traceback → lets you capture and format error messages. Eg : inside try/except, you can do traceback.print_exc() to print the full error stack.
import traceback

In [ ]:
'''Imports the ChatOpenAI class from LangChain (a library to work with LLMs like GPT).
    ChatOpenAI is used to connect with OpenAI’s models.
    Example: If you want GPT to answer, you’ll create a ChatOpenAI object.'''
from langchain.chat_models import ChatOpenAI

In [ ]:
'''dotenv is used to load sensitive data (like API keys) from a .env file instead of hardcoding them.
   load_dotenv() → automatically loads variables into your Python environment.'''

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

In [ ]:
'''Reads the environment variable named my-openkey into the Python variable KEY.
   KEY will now hold your OpenAI API key.'''

KEY=os.getenv("my-openkey")

In [ ]:
'''Creates an LLM object (chatbot connection).
   openai_api_key=KEY → uses your API key.
   model_name="gpt-3.5-turbo" → tells which OpenAI model to use.
   temperature=0.5 → controls creativity.
   0.0 → deterministic answers (less creativity).
   1.0 → more randomness.'''

llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [ ]:
# For to Inspect the object
llm

In [ ]:
# OpenAI → used for completion-style models like text-davinci-003. These expect plain text input and return plain text.
# Eg : If you want a chat with roles (system, user), you use ChatOpenAI.| If you just want “I give text → model gives text,” you use OpenAI.
from langchain.llms import OpenAI
# Lets you create templates for prompts where you can fill in blanks.
from langchain.prompts import PromptTemplate
# Combines a PromptTemplate + LLM into a single unit (called a chain).| You can pass inputs, and it automatically builds the prompt → sends to LLM → gives output.
from langchain.chains import LLMChain
# Lets you connect multiple chains together.| Output of Chain 1 → becomes input of Chain 2 → … and so on.
from langchain.chains import SequentialChain
# This is for tracking token usage and cost when you call the LLM.| Useful if you want to know how much money a request costs.
from langchain.callbacks import get_openai_callback
# A library for working with PDF files (reading text, pages, metadata).
import PyPDF2

In [ ]:
# RESPONSE_JSON is just a format guide → it tells the AI exactly how to structure quiz questions (with question, options, and correct answer).

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
# TEMPLATE is the instruction message for the AI.| the AI knows what to do and how to format the answer.

TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
'''PromptTemplate(...) wraps our template into a reusable object.
   You just give it the variables → it builds the correct instruction text for the AI.
   Later, it can be directly combined with LLMChain to generate quizzes.'''

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [ ]:
# now quiz_chain is ready to generate MCQs by just passing in variables | this is the first chain 
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path=r"/Users/muhammadsuleman/MCQGEN/Data.txt"

In [ ]:
file_path

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()
print(TEXT)

In [ ]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [ ]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [ ]:
print(f"Total Number of Tokens used in API call:{cb.total_tokens}")
print(f"Total Number of Tokens used in prompts:{cb.prompt_tokens}")
print(f"Number of Tokens generated by the model in its response:{cb.completion_tokens}")
print(f"Cost of the API call based on tokens used:{cb.total_cost}")

In [ ]:
response

In [ ]:
# .get("quiz") will return the value associated with the key "quiz".
# Converts a JSON string into a Python dictionary.

quiz=response.get("quiz")
quiz=json.loads(quiz)

In [ ]:
# This will hold all the formatted quiz questions.
quiz_table_data = []

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')